# Generate supported H5MD files

In [1]:
import contextlib
import os

import numpy as np
from ase.build import molecule
from ase.calculators.singlepoint import SinglePointCalculator
from ase.collections import s22

import znh5md

In [2]:
REMOVE_EXISTING = True

# Molecular Dynamics
- same system size
- velocities

In [3]:
if REMOVE_EXISTING:
    with contextlib.suppress(FileNotFoundError):
        os.remove("md.h5")

md_io = znh5md.IO("md.h5")
water = molecule("H2O")
for step in range(100):
    water.calc = SinglePointCalculator(
        water, energy=np.random.rand(), forces=np.random.rand(3, 3)
    )
    water.set_velocities(np.random.rand(3, 3))
    md_io.append(water)

# Dataset
- varying particle size
- all different kinds of `info` and `array` usages

In [4]:
if REMOVE_EXISTING:
    with contextlib.suppress(FileNotFoundError):
        os.remove("s22.h5")

frames = list(s22)
for atoms in frames:
    atoms.info["description"] = "Frame from the S22 dataset"
    atoms.calc = SinglePointCalculator(
        atoms, energy=np.random.rand(), forces=np.random.rand(len(atoms), 3)
    )
    atoms.calc.results["custom-energy"] = np.random.rand()
    atoms.calc.results["custom-forces"] = np.random.rand((len(atoms)), 3)
    atoms.arrays["meta"] = [{"species": x} for x in atoms.get_chemical_symbols()]

s22_io = znh5md.IO("s22.h5")
s22_io.extend(frames)

In [2]:
md_io = znh5md.IO("md.h5")
s22_io = znh5md.IO("s22.h5")

In [3]:
md_io[0].calc.results

{'forces': array([[0.12585768, 0.75307242, 0.17927539],
        [0.61802042, 0.41974151, 0.80937964],
        [0.85727344, 0.43807737, 0.54364063]]),
 'energy': 0.019128460438791794}

In [5]:
s22_io[0].info

{'custom-energy': 0.4116456362128119,
 'description': 'Frame from the S22 dataset'}